In [1]:
from datetime import datetime
from glob import glob
import os
import shutil
import numpy as np
import skimage.io
import skimage.transform
from skimage.transform import AffineTransform
from skimage.transform import warp
from skimage.transform import resize
from skimage.transform import SimilarityTransform

In [2]:
# 경로

# 메인경로
path_root = 'data/Caltech-101'

# train_org 모든 폴더와 파일을 삭제한다.
shutil.rmtree('data/Caltech-101/train_org/')

# train 폴더를 train_org로 변경한다.
os.rename('data/Caltech-101/train', 'data/Caltech-101/train_org')

# valid 폴더를 train_org로 변경한다.
os.rename('data/Caltech-101/valid', 'data/Caltech-101/valid_org')

# test 폴더를 test_org로 변경한다.
os.rename('data/Caltech-101/test', 'data/Caltech-101/test_org')


In [3]:
# 변환된 이미지가 담길 폴더를 생성한다.
if not os.path.exists('data/Caltech-101/train/all'):
    os.makedirs('data/Caltech-101/train/all')
    
if not os.path.exists('data/Caltech-101/valid/all'):
    os.makedirs('data/Caltech-101/valid/all')

if not os.path.exists('data/Caltech-101/test/all'):
    os.makedirs('data/Caltech-101/test/all')

In [4]:
# 홀드아웃용 폴더
for ho in range(0,2):
    for aug in range(5):
            if not os.path.exists(f'data/Caltech-101/train/{ho}/{aug}'):
                os.makedirs(f'data/Caltech-101/train/{ho}/{aug}')

            if not os.path.exists(f'data/Caltech-101/valid/{ho}/{aug}'):
                os.makedirs(f'data/Caltech-101/valid/{ho}/{aug}')

            if not os.path.exists(f'data/Caltech-101/test/{ho}/{aug}'):
                os.makedirs(f'data/Caltech-101/test/{ho}/{aug}')

In [5]:
# 데이터 변환을 위한 옵션값들
augmentation_params = {
    # 확장/축소 (가로세로 비율은 고정)
    'zoom_range' : (1,1),
    # 회전각도
    'rotation_range' : (-15,15),
    # 밀어 당기기
    'shear_range' : (-20,20),
    # 평행이동
    'translation_range' : (-30,30),
    # 반전
    'do_flip' : False,
    # 늘리기/ 줄이기 (가로세로 비율은 고정하지 않음)
    'allow_stretch' : 1.3
}

In [6]:
# 이미지 파일을 읽어오는 함수
def load(paths_train):
    # 이미지 데이터를 담을 리스트
    images = []
    # 이미지 이름
    imagenames = []
    # 결과종류
    labels = []
    
    # 파일의 수만큼 반복한다.
    for i, path in enumerate(paths_train):
        # 이미지를 읽어온다.
        a1 = skimage.io.imread(path)
        # 이미지를 224, 224 픽셀로 변환한다.
        image = resize(a1, (244,244))
        # 이미지 파일 이름을 가져온다.
        imagename = os.path.basename(path)
        # 결과 인덱스값
        label = os.path.basename(os.path.dirname(path))
        
        # print(image)
        # print(imagename)
        # print(label)

        # 리스트에 담는다.
        images.append(image)
        imagenames.append(imagename)
        labels.append(label)
        
    return images, imagenames, labels

In [8]:
# 이미지 워핑 : 이미지를 찌그러뜨린다.
def fast_warp (img, tf, output_shape=(50,50), mode='constant', order=1):
    m = tf.params
    a1 = warp(img, m, output_shape=output_shape, mode=mode, order=order)
    return a1

In [16]:
# 중앙점 위치를 이동시킨다.
def build_centering_transform(image_shape, target_shape=(50,50)):
    # 이차원 이미지 데이터라면
    if len(image_shape) == 2:
        rows, cols = image_shape
    
    # 컬러이미지
    else:
        rows, cols, _ = image_shape
        
    # 결과 데이터 가로 세로
    trows, tcols = target_shape
    
    # 이동한다.
    shift_x = (cols - tcols) / 2.0
    shift_y = (cols - tcols) / 2.0
    
    a1 = SimilarityTransform(translation=(shift_x, shift_y))
    return a1

In [10]:
# 중앙점을 타켓이미지 임의 위치로 이동시킨다.
def build_center_uncenter_transform(image_shape):
    center_shift = np.array([image_shape[1], image_shape[0]]) / 2.0 - 0.5
    tform_uncenter = SimilarityTransform(translation=-center_shift)
    tform_center = SimilarityTransform(translation=center_shift)
    return tform_uncenter, tform_center

In [13]:
# 이미지 확대 축소, 회전, 반전 등등
def build_transform(zoom=(1.0,1.0), rot=0, shear=0, trans=(0,0), flip=False):
    # 반전
    if flip:
        shear += 180
        rot += 180
    
    # 변환 각도를 라디언 값으로 변환한다.
    r_rad = np.deg2rad(rot)
    s_rad = np.deg2rad(shear)
    
    # 변환한다.
    tform_augment = AffineTransform(scale = (1/zoom[0], 1/zoom[1]),
                                            rotation = r_red, shear=s_red,
                                            translation=trans)
    return tform_augment

In [15]:
# 랜덤 변형
def random_transform(zoom_range, rotation_range, shear_range,
                    translation_range, do_flip=True, allow_stretch=False,
                    rng = np.random):
    # 이동 값
    shift_x = rng.uniform(*translation_range)
    shift_y = rng.uniform(*translation_range)
    translation = (shift_x, shift_y)
    
    # 회전값
    rotation_range = rng.uniform(*rotation_range)
    
    # 밀기값
    shear = rng.uniform(*shear_range)
    
    # do_flip(반전여부)가 True라면
    if do_flip:
        flip = (rng.randint(2)>0)
    else:
        flip = False
        
    # 확대 축소값
    log_zoom_range = [np.log(z) for z in zoom_range]
    
    # 위에서 구한 확대 축소값을 기반으로 확대 축소를 수행한다.
    # allow_stretch에 실수값이 들어올 경우 (확대 축소도를 정했을 경우)
    if isinstance(allow_stretch, float):
        # 확대 축소값을 구한다.
        log_stretch_range = [-np.log(allow_stretch), np.log(allow_stretch)]
        # 확대 축소 비율을 구한다.
        zoom = np.exp(rng.uniform(*log_zoom_range))
        # 늘리기 비율
        stretch = np.exp(rng.uniform(*log_stretch_range))
        # 늘릴 크기 값을 구한다.
        z_x = zoom * stretch
        z_y = zoom / stretch
    
    # allow_stretch에 True를 전달한 경우
    elif: 
        z_x = np.exp(rng.uniform(*log_zoom_range))
        z_y = np.exp(rng.uniform(*log_zoom_range))
        
    # allow_stretch에 False를 전달한 경우
    else:
        z_x = z_y = np.exp(ran.uniform(*log_zoom_range))
        
    # 변환 작업을 한다.
    a1 = build_transform((z_x, z_y), rotation, shear, translation, flip)
    return a1

In [17]:
# 위에서 만든 함수들을 모두 호출하는 함수
def perturb(img, augmentation_params, t_shape=(50,50), rng = np.random):
    tf_centering = build_centering_transform(img.shape, t_shape)
    tf_center, tf_uncenter = build_center_uncenter_transform(img, shape)
    tf_aug = random_transform(rng =rng, **augmentation_params)
    tf_aug = tf_uncenter + tf_aug + tf_center
    tf_aug = tf_centering + tf_aug
    warp_one = fast_warp(img,tf_aug, output_shape=t_shape, mode='constant')
    return warp_one.astype('float32')
    

In [7]:
paths_train = sorted(glob(f'{path_root}/train_org/0/*/*.jpg'))
images, imagenames, labels = load(paths_train)
imagenames[:5]

['image_0004.jpg',
 'image_0007.jpg',
 'image_0009.jpg',
 'image_0010.jpg',
 'image_0013.jpg']